In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-and-real-news-dataset/True.csv
/kaggle/input/fake-and-real-news-dataset/Fake.csv


Dataset link : https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset

In [2]:
import pandas as pd
import numpy as np

In [19]:
true = pd.read_csv('../input/fake-and-real-news-dataset/True.csv')
fake = pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv')
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['target'] = 0
true['target'] = 1
df = pd.concat([fake, true]).reset_index(drop = True)
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
new_df= df[20000:]
new_df.shape

(24898, 5)

In [11]:
new_df = new_df.dropna()
news = new_df.copy()
news.reset_index(inplace=True)

In [20]:
news.head(3)

,index,title,text,subject,date,target
0,34651,Attorney General Lynch seeks funds for expande...,WASHINGTON (Reuters) - A battle over gun owner...,politicsNews,"January 20, 2016",1
1,40276,Japan's Abe says won't delay tax hike unless b...,TOKYO (Reuters) - Japanese Prime Minister Shin...,worldnews,"October 22, 2017",1
2,5292,Stephen Colbert Goes Completely Off Script To...,The Stephen Colbert we love was in rare form W...,News,"July 28, 2016",0


In [13]:
X = news.drop('target', axis= 1)

In [14]:
y = news['target']

In [17]:
X.shape, y.shape

((24898, 5), (24898,))

In [18]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [21]:

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [22]:
### Vocabulary size
voc_size=5000

In [23]:
X['title'][1]

"Japan's Abe says won't delay tax hike unless big shock hits economy"

In [24]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
len(X)

24898

In [27]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]', ' ', X['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [31]:
print(len(corpus))
print(corpus[0])

24898
attorney gener lynch seek fund expand gun check


In [33]:
#one hot representation 
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0]

[18, 1781, 2814, 759, 515, 4304, 3606, 4291]

In [62]:
a= []
def max_one_hot_lenth():
    for i in range(len(onehot_repr)):
        a.append(len(onehot_repr[i]))
    return max(a)
max_one_hot_lenth()

35

In [63]:
#padding 
sent_length=35
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0   18
 1781 2814  759  515 4304 3606 4291]


In [64]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 35, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
len(embedded_docs),y.shape

(24898, (24898,))

In [66]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [67]:
X_final.shape, y_final.shape

((24898, 35), (24898,))

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [69]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Train on 16681 samples, validate on 8217 samples
Epoch 1/20
16681/16681 [==============================] - 14s 819us/sample - loss: 0.2946 - accuracy: 0.8797 - val_loss: 0.1853 - val_accuracy: 0.9250
Epoch 2/20
16681/16681 [==============================] - 12s 709us/sample - loss: 0.1366 - accuracy: 0.9477 - val_loss: 0.1658 - val_accuracy: 0.9318
Epoch 3/20
16681/16681 [==============================] - 12s 693us/sample - loss: 0.0966 - accuracy: 0.9646 - val_loss: 0.1754 - val_accuracy: 0.9310
Epoch 4/20
16681/16681 [==============================] - 11s 677us/sample - loss: 0.0674 - accuracy: 0.9762 - val_loss: 0.2048 - val_accuracy: 0.9276
Epoch 5/20
16681/16681 [==============================] - 11s 684us/sample - loss: 0.0497 - accuracy: 0.9828 - val_loss: 0.2401 - val_accuracy: 0.9282
Epoch 6/20
16681/16681 [==============================] - 12s 710us/sample - loss: 0.0357 - accuracy: 0.9886 - val_loss: 0.2684 - val_accuracy: 0.9263
Epoch 7/20
16681/16681 [=====================

In [70]:
y_pred=model.predict_classes(X_test)

In [71]:
from sklearn.metrics import confusion_matrix

In [72]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.919922112693197

In [73]:
# Adding dropout layer 
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 35, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [74]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Train on 16681 samples, validate on 8217 samples
Epoch 1/20
16681/16681 [==============================] - 14s 859us/sample - loss: 0.3067 - accuracy: 0.8620 - val_loss: 0.1851 - val_accuracy: 0.9247
Epoch 2/20
16681/16681 [==============================] - 12s 701us/sample - loss: 0.1489 - accuracy: 0.9429 - val_loss: 0.1768 - val_accuracy: 0.9292
Epoch 3/20
16681/16681 [==============================] - 12s 694us/sample - loss: 0.1150 - accuracy: 0.9568 - val_loss: 0.1757 - val_accuracy: 0.9294
Epoch 4/20
16681/16681 [==============================] - 12s 699us/sample - loss: 0.0883 - accuracy: 0.9682 - val_loss: 0.1825 - val_accuracy: 0.9317
Epoch 5/20
16681/16681 [==============================] - 12s 698us/sample - loss: 0.0704 - accuracy: 0.9743 - val_loss: 0.2013 - val_accuracy: 0.9241
Epoch 6/20
16681/16681 [==============================] - 12s 731us/sample - loss: 0.0543 - accuracy: 0.9814 - val_loss: 0.2251 - val_accuracy: 0.9261
Epoch 7/20
16681/16681 [=====================

In [75]:
y_pred=model.predict_classes(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9208957040282342